# Hemolysis

##### Load the CSV file into a DataFrame and reading the data

In [ ]:
import pandas as pd

df = pd.read_csv("Hemolytic toxicity.csv")
df

##### Data analysis

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

data = {
    'Nanoparticle concentration(mg/ml)': df['Nanoparticle concentration(mg/ml)'],
    'A': df.iloc[0, 1:].values,
    'B': df.iloc[1, 1:].values,
    'C': df.iloc[2, 1:].values,
    'D': df.iloc[3, 1:].values,
    'E': df.iloc[4, 1:].values,
    'F': df.iloc[5, 1:].values
}

rows = []
for conc, values in zip(data['Nanoparticle concentration(mg/ml)'], zip(data['A'], data['B'], data['C'], data['D'], data['E'], data['F'])):
    for group, value in zip(['A', 'B', 'C', 'D', 'E', 'F'], values):
        rows.append({
            'Nanoparticle concentration(mg/ml)': conc,
            'Group': group,
            'Values': value
        })

df2 = pd.DataFrame(rows)

# One-way ANOVA
model = ols('Values ~ C(Group)', data=df2).fit()

anova_table = sm.stats.anova_lm(model, typ=1)
print("ANOVA Table:")
print(anova_table)

# Tukey's test
tukey = pairwise_tukeyhsd(endog=df2['Values'], groups=df2['Group'], alpha=0.05)
print("\nTukey's HSD Test:")
print(tukey)

##### Plot the results

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from comparison import pairwise_comparison

df['Mean_Hemolysis'] = df[['Hemolysis1(%)', 'Hemolysis2(%)', 'Hemolysis3(%)']].mean(axis=1)

df['SEM'] = df[['Hemolysis1(%)', 'Hemolysis2(%)', 'Hemolysis3(%)']].sem(axis=1)

plt.figure(figsize=(6, 6))
bar_width = 0.6
x_ticks = np.arange(len(df))

plt.bar(x_ticks, df['Mean_Hemolysis'], width=bar_width, color='mediumorchid', edgecolor='indigo', linewidth=3.5)

errorbar_caps = plt.errorbar(x_ticks, df['Mean_Hemolysis'], yerr=[np.zeros(len(df['SEM'])), df['SEM']], capsize=10, color='indigo', ecolor='indigo', markersize=0, fmt='_', elinewidth=3)

for cap in errorbar_caps[1]:
    cap.set_markeredgewidth(3)
    cap.set_path_effects([path_effects.Stroke(linewidth=3), path_effects.Normal()])

font_properties = {'fontweight': 'bold', 'fontsize': 18, 'fontfamily': 'Arial'}
number_font_properties = {'fontweight': 'bold', 'fontsize': 18, 'fontfamily': 'Arial'}

plt.xlabel('Nanoparticle concentration (mg/ml)', **font_properties)
plt.ylabel('Hemolysis (%)', **font_properties)
plt.xticks(x_ticks, df['Nanoparticle concentration(mg/ml)'], rotation=45, rotation_mode='anchor', ha='right', **number_font_properties)
plt.yticks([0, 1, 2, 3, 4], **number_font_properties)

plt.tick_params(axis='both', which='both', length=10, width=3, labelsize=18)

ax = plt.gca()
ax.spines['left'].set_linewidth(3)
ax.spines['bottom'].set_linewidth(3)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

pairwise_comparison(4, 5, .0199, x_ticks, df['Mean_Hemolysis'].tolist(), yerr=df['SEM'].tolist(), dh=0.3, barh=0.02, fs=18, bold=True, line_width=2)
pairwise_comparison(3, 5, .0025, x_ticks, df['Mean_Hemolysis'].tolist(), yerr=df['SEM'].tolist(), dh=0.14, barh=0.02, fs=18, bold=True, line_width=2)
pairwise_comparison(1, 5, .0007, x_ticks, df['Mean_Hemolysis'].tolist(), yerr=df['SEM'].tolist(), dh=0.03, barh=0.02, fs=18, bold=True, line_width=2)

plt.ylim(bottom=0)
plt.savefig('Hemolytic_toxicity_graph.tif', dpi=600, format='tif', bbox_inches='tight')
plt.show()